This is a notebook to time how long it takes to create the texture features (1st and 2nd order) in the way they are implemented in the classification. 

In [2]:
import os
import time
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from shapely.geometry import Polygon

from skimage.feature import graycomatrix, graycoprops

from skimage.morphology import disk
from skimage.filters.rank import entropy

from scipy.ndimage import convolve as conf2D

In [3]:
box_carp = Polygon( [
            [-119.5144546684841,34.39274397377842],
            [-119.5144546684841,34.39193499538564],
            [-119.51344346289721,34.39193499538564],
            [-119.51344346289721,34.39274397377842],
            [-119.5144546684841,34.39274397377842]
          ])
itemid_carp = 'ca_m_3411936_se_11_060_20200521'

# I already pre-checled this image is at least 150x150 pixels
raster = sr.rioxr_from_itemid(itemid_carp, box_carp, "EPSG:4326")[:,0:150,0:150]

# the computations on the array left by leaving a margin of 15 pixels around all edges
# this will give enough space to compute textures across all window sizes
buffer = 15
raster

<xarray.DataArray (band: 4, y: 150, x: 150)>
[90000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 2.688e+05 2.688e+05 ... 2.689e+05 2.689e+05
  * y            (y) float64 3.809e+06 3.809e+06 ... 3.808e+06 3.808e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    TIFFTAG_IMAGEDESCRIPTION:  OrthoVista
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    TIFFTAG_SOFTWARE:          Trimble Germany GmbH
    TIFFTAG_XRESOLUTION:       1
    TIFFTAG_YRESOLUTION:       1
    scale_factor:              1.0
    add_offset:                0.0
    _FillValue:                0
    datetime:                  2020-05-21 00:00:00+00:00

In [4]:
window_radii = [1,2,3,4,5,6,7,8,9,10,11,12,13]

# --------------------------------------------
# TIME FOR GLCM FEATURE CREATION
times_glcm = []

# parameters for GLCM 
distances = [1]     
angles = [0, np.pi/2] # East, North

y_len = raster.shape[1]
x_len = raster.shape[2]

for window_r in window_radii:

    # --------------------------
    # create arrays that will hold GLCM features for both angles
    contrast = {angles[0]: np.zeros((4,y_len - buffer*2, x_len - buffer*2)), 
                angles[1]: np.zeros((4,y_len - buffer*2, x_len - buffer*2))}

    correlation = {angles[0]: np.zeros((4,y_len - buffer*2, x_len - buffer*2)), 
                   angles[1]: np.zeros((4,y_len - buffer*2, x_len - buffer*2))}

    # --------------------------
    # START TIMING
    t0 = time.time()

    # only compute for pixels within inner image (outside buffer)
    for y in range(16,136):
        for x in range(16,136):
            # window has shape (4, 2*window_r +1, 2*window_r +1)
            window = raster[:, y-window_r:y+window_r+1, x-window_r:x+window_r+1]

            # --------------------------
            # *** CHANGE: graycomatrix can compute both angles simultaneously ***
            # *** see documentation ***
            for angle in angles:   
                for band in range(4):
                    glcm = graycomatrix(window[band],  # can only calculate one band at a time
                                distances=distances,
                                angles=[angle])

                    # extract contrast and correlation from GLCM
                    contrast[angle][band,y-16,x-16] = graycoprops(glcm, 'contrast')[0,0]
                    correlation[angle][band,y-16,x-16] = graycoprops(glcm, 'correlation')[0,0]
    # --------------------------
    # FINISH TIMING
    times.append(time.time() - t0)
    # save intermediate results in case of crashing
    np.savetxt('glcm_times.txt',np.array(times),fmt='%.10f')
    print('finished ', window_r)

# --------------------------
df = pd.DataFrame({'time':times, 'radius':window_radii})
df.to_csv('times_glcm_6to13.csv', index=False)

In [ ]:
## PLOT LAST RESULT
# fig, ax = plt.subplots(2,4,figsize=(20, 20))  

# for i,angle in zip(range(2),angles):
#     for band in range(4):
#         ax[i,band].imshow(contrast[angle][band])
# plt.show()

In [8]:
inner_raster = raster[:,16:136,16:136]
window_radii = [1,2,3,4,5,6,7,8,9,10,11,12,13]

# --------------------------------------------
# TIME AVG FEATURE CREATION
times_avg = []

for window_r in window_radii:
    box_side = window_r*2+1
    # START TIMING    
    t0 = time.time()        

    for band in range(4): 
        w = np.ones(box_side*box_side).reshape(box_side,box_side)      
        avgs = conf2D(inner_raster[band], 
                 weights=w,
                 mode='constant', # how the input array is extended beyond its boundaries
                 output='int64')
        avgs = avgs/(box_side**2)
    

    # FINISH TIMING
    times_avg.append(time.time() - t0)

    
# --------------------------------------------
# TIME ENTROPY FEATURE CREATION

times_entr = []

for window_r in window_radii:
    
    # START TIMING
    t0 = time.time()
    for band in range(4):
        entropy(inner_raster[band], disk(window_r)) 

    # FINISH TIMING
    times_entr.append(time.time() - t0)


# --------------------------------------------
df2 = pd.DataFrame({'radius': window_radii,
                    'time_avg': times_avg,
                    'time_entr': times_entr})
df2.to_csv('avg_entr_texture_times.csv',index=False)
df2